In [12]:
import numpy as np
import librosa
import tensorflow as tf

def predict_tone(file_path, model_path="tone_recognition_model.h5"):
    # Načtení natrénovaného modelu
    model = tf.keras.models.load_model(model_path)
    
    # Parametry (musí odpovídat trénovacímu skriptu)
    SAMPLE_RATE = 22050
    N_FFT = 2048
    HOP_LENGTH = 512
    N_MELS = 128
    IMG_SIZE = (128, 128)
    
    # Načtení a předzpracování audio souboru
    signal, sr = librosa.load(file_path, sr=SAMPLE_RATE)
    mel_spec = librosa.feature.melspectrogram(y=signal, sr=sr, n_fft=N_FFT, 
                                              hop_length=HOP_LENGTH, n_mels=N_MELS)
    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)
    
    # Přizpůsobení tvaru pro model
    mel_spec_db = np.expand_dims(mel_spec_db, axis=-1)  # Přidání kanálu
    mel_spec_db = tf.image.resize(mel_spec_db, IMG_SIZE).numpy()
    input_data = np.expand_dims(mel_spec_db, axis=0)    # Přidání batch dimenze
    
    # Predikce
    predictions = model.predict(input_data)
    predicted_label = np.argmax(predictions[0])
    
    # Převedení labelu na název tónu
    label_to_tone = {0: "C4", 1: "D4", 2: "E4"}
    return label_to_tone[predicted_label]

# Příklad použití
if __name__ == "__main__":
    file_path = "testing/c4_testing/testing_2.wav"  # Zde zadejte cestu k vašemu .wav souboru
    predicted_tone = predict_tone(file_path)
    print(f"Predikovaný tón: {predicted_tone}")

1/1 [==============================] - 0s 160ms/step
Predikovaný tón: C4


In [38]:
import numpy as np
import librosa
import tensorflow as tf

def predict_tone(file_path, model_path="tone_recognition_multi_label.h5"):
    model = tf.keras.models.load_model(model_path)
    
    # Parametry
    SAMPLE_RATE = 22050
    N_FFT = 2048
    HOP_LENGTH = 512
    N_MELS = 128
    IMG_SIZE = (128, 128)
    
    # Načtení a zpracování audia
    signal, sr = librosa.load(file_path, sr=SAMPLE_RATE)
    mel_spec = librosa.feature.melspectrogram(y=signal, sr=sr, n_fft=N_FFT,
                                              hop_length=HOP_LENGTH, n_mels=N_MELS)
    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)
    
    # Příprava vstupu pro model
    mel_spec_db = np.expand_dims(mel_spec_db, axis=-1)
    mel_spec_db = tf.image.resize(mel_spec_db, IMG_SIZE).numpy()
    input_data = np.expand_dims(mel_spec_db, axis=0)
    
    # Predikce s detaily jistoty
    predictions = model.predict(input_data)
    predicted_label = np.argmax(predictions[0])
    confidence = np.max(predictions[0]) * 100  # Procentuální jistota
    
    label_to_tone = {0: "C4", 1: "D4", 2: "E4", 3: "F4", 4: "G4", 5: "A4", 6: "B4"}
    print("Detailní jistoty:")
    for tone, conf in zip(["C4", "D4", "E4","F4", "G4", "A4", "B4"], predictions[0]):
        print(f"{tone}: {conf*100:.2f}%")
    return label_to_tone[predicted_label], confidence

# Příklad použití
if __name__ == "__main__":
    file_path = "nwm/temp_edit_2.wav"
    predicted_tone, confidence = predict_tone(file_path)
    print(f"Predikovaný tón: {predicted_tone} | Jistota: {confidence:.2f}%")

1/1 [==============================] - 0s 116ms/step
Detailní jistoty:
C4: 100.00%
D4: 0.00%
E4: 0.00%
F4: 0.01%
G4: 0.00%
A4: 0.00%
B4: 0.00%
Predikovaný tón: C4 | Jistota: 100.00%


In [17]:
import numpy as np
import librosa
import tensorflow as tf
import sounddevice as sd  # Pro nahrávání zvuku
import soundfile as sf    # Pro uložení nahraného zvuku

def predict_tone(file_path, model_path="tone_recognition_model.h5"):
    model = tf.keras.models.load_model(model_path)
    
    # Parametry
    SAMPLE_RATE = 22050
    N_FFT = 2048
    HOP_LENGTH = 512
    N_MELS = 128
    IMG_SIZE = (128, 128)
    
    # Načtení a zpracování audia
    signal, sr = librosa.load(file_path, sr=SAMPLE_RATE)
    mel_spec = librosa.feature.melspectrogram(y=signal, sr=sr, n_fft=N_FFT,
                                              hop_length=HOP_LENGTH, n_mels=N_MELS)
    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)
    
    # Příprava vstupu pro model
    mel_spec_db = np.expand_dims(mel_spec_db, axis=-1)
    mel_spec_db = tf.image.resize(mel_spec_db, IMG_SIZE).numpy()
    input_data = np.expand_dims(mel_spec_db, axis=0)
    
    # Predikce
    predictions = model.predict(input_data)
    predicted_label = np.argmax(predictions[0])
    confidence = np.max(predictions[0]) * 100

    label_to_tone = {0: "C4", 1: "D4", 2: "E4", 3: "F4", 4: "G4", 5: "A4", 6: "B4"}
    print("Detailní jistoty:")
    for tone, conf in zip(["C4", "D4", "E4","F4", "G4", "A4", "B4"], predictions[0]):
        print(f"{tone}: {conf*100:.2f}%")
    return label_to_tone[predicted_label], confidence

def record_audio(duration=1, sample_rate=22050, filename="temp.wav"):
    """Nahrává zvuk z mikrofonu a uloží do souboru."""
    print(f"Nahrávám {duration} vteřin...")
    audio = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=1)
    sd.wait()  # Čeká na dokončení nahrávání
    sf.write(filename, audio, sample_rate)
    print(f"Uloženo do {filename}")

if __name__ == "__main__":
    # Nahrání zvuku
    record_audio(duration=2)  # Nahrává 1 vteřinu
    
    # Predikce
    predicted_tone, confidence = predict_tone("temp.wav")
    print(f"Predikovaný tón: {predicted_tone} | Jistota: {confidence:.2f}%")

Nahrávám 2 vteřin...
Uloženo do temp.wav
1/1 [==============================] - 0s 125ms/step
Detailní jistoty:
C4: 0.03%
D4: 0.00%
E4: 0.00%
F4: 0.01%
G4: 0.00%
A4: 99.96%
B4: 0.00%
Predikovaný tón: A4 | Jistota: 99.96%


In [23]:
import numpy as np
import librosa
import tensorflow as tf
import sounddevice as sd  # Pro nahrávání a přehrávání
import soundfile as sf    # Pro práci se soubory

def predict_tone(file_path, model_path="tone_recognition_multi_label.h5"):
    model = tf.keras.models.load_model(model_path)
    
    # Parametry
    SAMPLE_RATE = 22050
    N_FFT = 2048
    HOP_LENGTH = 512
    N_MELS = 128
    IMG_SIZE = (128, 128)
    
    # Načtení a zpracování audia
    signal, sr = librosa.load(file_path, sr=SAMPLE_RATE)
    mel_spec = librosa.feature.melspectrogram(y=signal, sr=sr, n_fft=N_FFT,
                                              hop_length=HOP_LENGTH, n_mels=N_MELS)
    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)
    
    # Příprava vstupu pro model
    mel_spec_db = np.expand_dims(mel_spec_db, axis=-1)
    mel_spec_db = tf.image.resize(mel_spec_db, IMG_SIZE).numpy()
    input_data = np.expand_dims(mel_spec_db, axis=0)
    
    # Predikce
    predictions = model.predict(input_data)
    predicted_label = np.argmax(predictions[0])
    confidence = np.max(predictions[0]) * 100

    label_to_tone = {0: "C4", 1: "D4", 2: "E4", 3: "F4", 4: "G4", 5: "A4", 6: "B4"}
    print("Detailní jistoty:")
    for tone, conf in zip(["C4", "D4", "E4","F4", "G4", "A4", "B4"], predictions[0]):
        print(f"{tone}: {conf*100:.2f}%")
    return label_to_tone[predicted_label], confidence

def record_audio(duration=1, sample_rate=22050, filename="temp.wav"):
    """Nahrává zvuk z mikrofonu a uloží do souboru."""
    print(f"Nahrávám {duration} vteřin...")
    audio = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=1)
    sd.wait()  # Čeká na dokončení nahrávání
    sf.write(filename, audio, sample_rate)
    print(f"Uloženo do {filename}")
    return audio  # Vracíme nahrané audio pro případné přehrání

def play_audio(file_path):
    """Přehrává audio ze souboru."""
    audio, sr = sf.read(file_path)
    print("Přehrávám audio...")
    sd.play(audio, sr)
    sd.wait()  # Čeká na dokončení přehrávání
    print("Přehrávání dokončeno")

if __name__ == "__main__":
    # Nahrání zvuku
    record_audio(duration=3)  # Nahrává 1 vteřinu
    
    # Přehrání pro kontrolu
    play_audio("temp.wav")
    
    # Predikce
    predicted_tone, confidence = predict_tone("temp.wav")
    print(f"Predikovaný tón: {predicted_tone} | Jistota: {confidence:.2f}%")

Nahrávám 3 vteřin...
Uloženo do temp.wav
Přehrávám audio...
Přehrávání dokončeno
1/1 [==============================] - 0s 107ms/step
Detailní jistoty:
C4: 0.10%
D4: 0.00%
E4: 0.00%
F4: 0.89%
G4: 0.00%
A4: 30.51%
B4: 0.00%
Predikovaný tón: A4 | Jistota: 30.51%


In [36]:
import numpy as np
import librosa
import tensorflow as tf
import sounddevice as sd  # Pro nahrávání a přehrávání
import soundfile as sf    # Pro práci se soubory

def predict_tone(file_path, model_path="tone_recognition_multi_label.h5"):
    model = tf.keras.models.load_model(model_path)
    
    # Parametry
    SAMPLE_RATE = 22050
    N_FFT = 2048
    HOP_LENGTH = 512
    N_MELS = 128
    IMG_SIZE = (128, 128)
    
    # Načtení a zpracování audia
    signal, sr = librosa.load(file_path, sr=SAMPLE_RATE)
    mel_spec = librosa.feature.melspectrogram(y=signal, sr=sr, n_fft=N_FFT,
                                              hop_length=HOP_LENGTH, n_mels=N_MELS)
    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)
    
    # Příprava vstupu pro model
    mel_spec_db = np.expand_dims(mel_spec_db, axis=-1)
    mel_spec_db = tf.image.resize(mel_spec_db, IMG_SIZE).numpy()
    input_data = np.expand_dims(mel_spec_db, axis=0)
    
    # Predikce
    predictions = model.predict(input_data)
    predicted_label = np.argmax(predictions[0])
    confidence = np.max(predictions[0]) * 100

    label_to_tone = {0: "C4", 1: "D4", 2: "E4", 3: "F4", 4: "G4", 5: "A4", 6: "B4"}
    print("Detailní jistoty:")
    for tone, conf in zip(["C4", "D4", "E4","F4", "G4", "A4", "B4"], predictions[0]):
        print(f"{tone}: {conf*100:.2f}%")
    return label_to_tone[predicted_label], confidence



def adjust_volume(audio, gain=4.0):
    """Zesílí audio signál a ohlídá clipping."""
    amplified = audio * gain
    
    # Oříznutí hodnot mimo rozsah [-1.0, 1.0] pro prevenci zkreslení
    amplified = np.clip(amplified, -1.0, 1.0)
    return amplified

def record_audio(duration=1, sample_rate=22050, filename="temp.wav", gain=2.0):
    """Nahrává zvuk a automaticky zesílí hlasitost."""
    print(f"Nahrávám {duration} vteřin...")
    audio = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=1)
    sd.wait()
    
    # Zesílení a normalizace
    audio = adjust_volume(audio, gain)
    
    sf.write(filename, audio, sample_rate)
    print(f"Uloženo do {filename} (zesílení: {gain}x)")
    return audio

def play_audio(file_path, gain=1.0):
    """Přehrává audio s možností zesílení."""
    audio, sr = sf.read(file_path)
    audio = adjust_volume(audio, gain)  # Zesílení při přehrávání
    print(f"Přehrávám audio (zesílení: {gain}x)...")
    sd.play(audio, sr)
    sd.wait()

if __name__ == "__main__":
    # Nahrání se zesílením 3x
    record_audio(duration=3, gain=3.0)
    
    # Přehrání se zesílením 2x (pro ještě větší hlasitost)
    play_audio("temp.wav", gain=2.0)
    
    # Predikce
    predicted_tone, confidence = predict_tone("temp.wav")
    print(f"Predikovaný tón: {predicted_tone} | Jistota: {confidence:.2f}%")

Nahrávám 3 vteřin...
Uloženo do temp.wav (zesílení: 3.0x)
Přehrávám audio (zesílení: 2.0x)...
1/1 [==============================] - 0s 108ms/step
Detailní jistoty:
C4: 0.28%
D4: 0.00%
E4: 0.00%
F4: 3.85%
G4: 0.00%
A4: 0.97%
B4: 0.00%
Predikovaný tón: F4 | Jistota: 3.85%


In [49]:
import numpy as np
import librosa
import tensorflow as tf
import sounddevice as sd
import soundfile as sf

def adjust_volume(audio, gain=2.0):
    amplified = audio * gain
    return np.clip(amplified, -1.0, 1.0)

def record_audio(duration=1, sample_rate=22050, filename="temp.wav", gain=2.0):
    print(f"Nahrávám {duration} vteřin...")
    audio = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=1)
    sd.wait()
    
    # Zesílení a převod na 1D pole
    audio = adjust_volume(audio.squeeze(), gain)
    
    # Detekce onsetů s vašimi parametry
    onsets = librosa.onset.onset_detect(
        y=audio,
        sr=sample_rate,
        units='time',
        hop_length=512,
        pre_max=3,
        post_max=3,
        pre_avg=3,
        post_avg=3,
        delta=0.3,
        wait=0
    )
    
    # Oříznutí na první onset
    if len(onsets) > 0:
        start_time = onsets[0]
        start_sample = int(start_time * sample_rate)
        trimmed_audio = audio[start_sample:]
        print(f"Audio oříznuto na onset v čase {start_time:.2f}s")
    else:
        trimmed_audio = audio
        print("Onset nebyl detekován. Používám celou nahrávku.")
    
    # Uložení oříznutého audia
    sf.write(filename, trimmed_audio, sample_rate)
    return trimmed_audio

def predict_tone(file_path, model_path="tone_recognition_multi_label.h5"):
    model = tf.keras.models.load_model(model_path)
    
    # Načtení a zpracování
    signal, sr = librosa.load(file_path, sr=22050)
    mel_spec = librosa.feature.melspectrogram(y=signal, sr=sr, n_fft=2048, hop_length=512, n_mels=128)
    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)
    
    # Příprava vstupu
    mel_spec_db = np.expand_dims(mel_spec_db, axis=-1)
    mel_spec_db = tf.image.resize(mel_spec_db, (128, 128)).numpy()
    input_data = np.expand_dims(mel_spec_db, axis=0)
    
    # Predikce
    predictions = model.predict(input_data)
    predicted_label = np.argmax(predictions[0])
    confidence = np.max(predictions[0]) * 100

    label_to_tone = {0: "C4", 1: "D4", 2: "E4", 3: "F4", 4: "G4", 5: "A4", 6: "B4"}
    print("\nDetailní jistoty:")
    for tone, conf in zip(label_to_tone.values(), predictions[0]):
        print(f"{tone}: {conf*100:.2f}%")
    return label_to_tone[predicted_label], confidence

def play_audio(file_path):
    audio, sr = sf.read(file_path)
    print("\nPřehrávám oříznuté audio...")
    sd.play(audio, sr)
    sd.wait()

if __name__ == "__main__":
    # Nahrání s agresivním zesílením
    record_audio(duration=3, gain=3.0)
    
    # Kontrola a predikce
    play_audio("temp.wav")
    predicted_tone, confidence = predict_tone("temp.wav")
    print(f"\nPredikovaný tón: {predicted_tone} | Jistota: {confidence:.2f}%")

Nahrávám 3 vteřin...
Audio oříznuto na onset v čase 0.07s

Přehrávám oříznuté audio...
1/1 [==============================] - 0s 101ms/step

Detailní jistoty:
C4: 0.00%
D4: 0.00%
E4: 0.00%
F4: 0.00%
G4: 0.00%
A4: 99.78%
B4: 0.00%

Predikovaný tón: A4 | Jistota: 99.78%


In [55]:
import numpy as np
import librosa
import tensorflow as tf
import sounddevice as sd
from queue import Queue

# Nastavení parametrů
SAMPLE_RATE = 22050
BUFFER_DURATION = 1.0  # Délka bufferu v sekundách
ONSET_THRESHOLD = 0.3  # Citlivost detekce začátku tónu

# Načtení modelu
model = tf.keras.models.load_model("tone_recognition_multi_label.h5")

# Fronta pro přenos dat mezi streamem a hlavním vláknem
audio_queue = Queue()

def audio_callback(indata, frames, time, status):
    """Funkce volaná pro každý nový audio blok."""
    audio_queue.put(indata.copy())

def process_audio(buffer):
    """Zpracuje audio buffer a vrátí predikovaný tón."""
    # Generování mel spektrogramu
    mel_spec = librosa.feature.melspectrogram(
        y=buffer,
        sr=SAMPLE_RATE,
        n_fft=2048,
        hop_length=512,
        n_mels=128
    )
    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)
    
    # Příprava pro model
    mel_spec_db = np.expand_dims(mel_spec_db, axis=-1)
    mel_spec_db = tf.image.resize(mel_spec_db, (128, 128)).numpy()
    input_data = np.expand_dims(mel_spec_db, axis=0)
    
    # Predikce
    predictions = model.predict(input_data, verbose=0)
    return np.argmax(predictions[0]), np.max(predictions[0])

def detect_onset(buffer):
    """Detekuje onset v aktuálním bufferu."""
    return librosa.onset.onset_detect(
        y=buffer,
        sr=SAMPLE_RATE,
        units='samples',
        hop_length=512,
        delta=ONSET_THRESHOLD,
        pre_max=3,
        post_max=3
    )

# Slovník pro překlad labelů
label_to_tone = {0: "C4", 1: "D4", 2: "E4", 3: "F4", 4: "G4", 5: "A4", 6: "B4"}

# Spuštění streamu
with sd.InputStream(
    callback=audio_callback,
    channels=1,
    samplerate=SAMPLE_RATE,
    blocksize=int(SAMPLE_RATE * 0.1)  # 100ms bloky
):
    print("=== REALTIME DETEKCE TÓNŮ ===")
    print("Hrajte tón na mikrofon... (Ctrl+C pro ukončení)")
    
    buffer = np.array([], dtype=np.float32)
    is_playing = False
    
    while True:
        # Načtení dat z fronty
        while not audio_queue.empty():
            new_data = audio_queue.get().squeeze()
            buffer = np.concatenate([buffer, new_data])
        
        # Udržujeme buffer o délce BUFFER_DURATION
        if len(buffer) > SAMPLE_RATE * BUFFER_DURATION:
            buffer = buffer[-int(SAMPLE_RATE * BUFFER_DURATION):]
        
        # Detekce onsetu
        onsets = detect_onset(buffer)
        
        if len(onsets) > 0 and not is_playing:
            # Zpracování od posledního onsetu
            start_sample = onsets[-1]
            segment = buffer[start_sample:]
            
            if len(segment) > 2048:  # Minimální délka pro analýzu
                is_playing = True
                predicted_label, confidence = process_audio(segment)
                tone = label_to_tone[predicted_label]
                print(f"\nDetekován tón: {tone} | Jistota: {confidence*100:.1f}%")
                is_playing = False

=== REALTIME DETEKCE TÓNŮ ===
Hrajte tón na mikrofon... (Ctrl+C pro ukončení)

Detekován tón: A4 | Jistota: 91.5%

Detekován tón: A4 | Jistota: 90.8%

Detekován tón: A4 | Jistota: 96.5%

Detekován tón: A4 | Jistota: 94.4%

Detekován tón: A4 | Jistota: 90.3%

Detekován tón: A4 | Jistota: 90.0%

Detekován tón: A4 | Jistota: 89.6%

Detekován tón: A4 | Jistota: 95.7%

Detekován tón: A4 | Jistota: 97.7%

Detekován tón: A4 | Jistota: 97.5%

Detekován tón: A4 | Jistota: 97.3%

Detekován tón: C4 | Jistota: 92.1%

Detekován tón: A4 | Jistota: 35.9%

Detekován tón: A4 | Jistota: 76.4%

Detekován tón: A4 | Jistota: 65.7%

Detekován tón: C4 | Jistota: 98.2%

Detekován tón: C4 | Jistota: 98.2%

Detekován tón: C4 | Jistota: 99.6%

Detekován tón: C4 | Jistota: 99.9%

Detekován tón: C4 | Jistota: 100.0%

Detekován tón: C4 | Jistota: 100.0%

Detekován tón: C4 | Jistota: 100.0%

Detekován tón: C4 | Jistota: 100.0%

Detekován tón: A4 | Jistota: 98.9%

Detekován tón: A4 | Jistota: 97.6%

Detekován tón: A4

KeyboardInterrupt: 

In [5]:
import numpy as np
import librosa
import tensorflow as tf
import sounddevice as sd
from queue import Queue
import time

# Nastavení parametrů
SAMPLE_RATE = 22050
BUFFER_DURATION = 1.0
ONSET_THRESHOLD = 0.3
MIN_DELAY = 0.5  # Minimální prodleva mezi detekcemi (v sekundách)

# Načtení modelu
model = tf.keras.models.load_model("tone_recognition_multi_label.h5")

audio_queue = Queue()
last_detection_time = 0
label_to_tone = {0: "C4", 1: "D4", 2: "E4", 3: "F4", 4: "G4", 5: "A4", 6: "B4"}

def audio_callback(indata, frames, time, status):
    audio_queue.put(indata.copy())

def process_audio(buffer):
    mel_spec = librosa.feature.melspectrogram(y=buffer, sr=SAMPLE_RATE, n_fft=2048, hop_length=512, n_mels=128)
    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)
    mel_spec_db = np.expand_dims(mel_spec_db, axis=-1)
    mel_spec_db = tf.image.resize(mel_spec_db, (128, 128)).numpy()
    input_data = np.expand_dims(mel_spec_db, axis=0)
    predictions = model.predict(input_data, verbose=0)
    return np.argmax(predictions[0]), np.max(predictions[0])

def detect_onset(buffer):
    return librosa.onset.onset_detect(
        y=buffer, sr=SAMPLE_RATE, units='samples', hop_length=512,
        delta=ONSET_THRESHOLD, pre_max=3, post_max=3
    )

with sd.InputStream(callback=audio_callback, channels=1, samplerate=SAMPLE_RATE, blocksize=int(SAMPLE_RATE*0.1)):
    print("=== REALTIME DETEKCE TÓNŮ ===")
    print("Hrajte tón na mikrofon... (Ctrl+C pro ukončení)\n")
    print("Poslední detekce:".ljust(40) + "Čekám na tón...\r", end='')
    
    buffer = np.array([], dtype=np.float32)
    
    while True:
        # Aktualizace bufferu
        while not audio_queue.empty():
            buffer = np.concatenate([buffer, audio_queue.get().squeeze()])
        
        # Omezíme délku bufferu
        buffer = buffer[-int(SAMPLE_RATE*BUFFER_DURATION):] if len(buffer) > SAMPLE_RATE*BUFFER_DURATION else buffer
        
        # Detekce onsetů
        onsets = detect_onset(buffer)
        current_time = time.time()
        
        if len(onsets) > 0 and (current_time - last_detection_time) > MIN_DELAY:
            start_sample = onsets[-1]
            segment = buffer[start_sample:]
            
            if len(segment) > 2048:
                predicted_label, confidence = process_audio(segment)
                tone = label_to_tone[predicted_label]
                output = f"{tone} ({confidence*100:.1f}%)".ljust(20)
                print(f"Poslední detekce: {output} | Čas: {time.strftime('%H:%M:%S')}\r", end='')
                last_detection_time = current_time

ValueError: Name tf.RaggedTensorSpec has already been registered for class tensorflow.python.ops.ragged.ragged_tensor.RaggedTensorSpec.

In [2]:
# SUPER FUNGUJE!!! NEJLEPŠÍ ZATÍM!
import numpy as np
import librosa
import tensorflow as tf
import sounddevice as sd
from queue import Queue
import time

# Nastavení parametrů
SAMPLE_RATE = 22050
BUFFER_DURATION = 1.0
ONSET_THRESHOLD = 0.3
MIN_DELAY = 0.5  # Minimální prodleva mezi detekcemi (v sekundách)

# Načtení modelu
model = tf.keras.models.load_model("tone_recognition_multi_label.h5")

audio_queue = Queue()
last_detection_time = 0
label_to_tone = label_to_tone = {
    0: "C4", 
    1: "D4", 
    2: "E4", 
    3: "F4", 
    4: "G4", 
    5: "A4", 
    6: "B4", 
    7: "C5", 
    8: "D5", 
    9: "E5", 
    10: "F5", 
    11: "G5", 
    12: "A5", 
    13: "B5", 
    14: "C6", 
    15: "D6", 
    16: "E6"
}

def audio_callback(indata, frames, time, status):
    audio_queue.put(indata.copy())

def process_audio(buffer):
    mel_spec = librosa.feature.melspectrogram(y=buffer, sr=SAMPLE_RATE, n_fft=2048, hop_length=512, n_mels=128)
    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)
    mel_spec_db = np.expand_dims(mel_spec_db, axis=-1)
    mel_spec_db = tf.image.resize(mel_spec_db, (128, 128)).numpy()
    input_data = np.expand_dims(mel_spec_db, axis=0)
    predictions = model.predict(input_data, verbose=0)
    return np.argmax(predictions[0]), np.max(predictions[0])

def detect_onset(buffer):
    return librosa.onset.onset_detect(
        y=buffer, sr=SAMPLE_RATE, units='samples', hop_length=512,
        delta=ONSET_THRESHOLD, pre_max=3, post_max=3
    )

with sd.InputStream(callback=audio_callback, channels=1, samplerate=SAMPLE_RATE, blocksize=int(SAMPLE_RATE*0.1)):
    print("=== REALTIME DETEKCE TÓNŮ ===")
    print("Hrajte tón na mikrofon... (Ctrl+C pro ukončení)\n")
    print("Poslední detekce:".ljust(40) + "Čekám na tón...\r", end='')
    
    buffer = np.array([], dtype=np.float32)
    
    while True:
        # Aktualizace bufferu
        while not audio_queue.empty():
            buffer = np.concatenate([buffer, audio_queue.get().squeeze()])
        
        # Omezíme délku bufferu
        buffer = buffer[-int(SAMPLE_RATE*BUFFER_DURATION):] if len(buffer) > SAMPLE_RATE*BUFFER_DURATION else buffer
        
        # Detekce onsetů
        onsets = detect_onset(buffer)
        current_time = time.time()
        
        if len(onsets) > 0 and (current_time - last_detection_time) > MIN_DELAY:
            start_sample = onsets[-1]
            segment = buffer[start_sample:]
            
            if len(segment) > 2048:
                predicted_label, confidence = process_audio(segment)
                tone = label_to_tone[predicted_label]
                output = f"{tone} ({confidence*100:.1f}%)".ljust(20)
                print(f"Poslední detekce: {output} | Čas: {time.strftime('%H:%M:%S')}\r", end='')
                last_detection_time = current_time

C:\Users\Dave\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


=== REALTIME DETEKCE TÓNŮ ===
Hrajte tón na mikrofon... (Ctrl+C pro ukončení)

Poslední detekce:                       Čekám na tón...

C:\Users\Dave\AppData\Local\Programs\Python\Python312\Lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=0
  warnings.warn(


Poslední detekce: E6 (0.4%)            | Čas: 22:31:35

Exception ignored in: <function WeakKeyDictionary.__init__.<locals>.remove at 0x00000169F99CF560>
Traceback (most recent call last):
  File "C:\Users\Dave\AppData\Local\Programs\Python\Python312\Lib\weakref.py", line 369, in remove
    def remove(k, selfref=ref(self)):

KeyboardInterrupt: 


Poslední detekce: A4 (2.5%)            | Čas: 22:31:38


KeyboardInterrupt



In [7]:
import numpy as np
import librosa
import tensorflow as tf
import sounddevice as sd
from queue import Queue
import time
from joblib import load

# Konfigurace
SAMPLE_RATE = 22050
BUFFER_DURATION = 1.5
MIN_SEGMENT_LENGTH = 2048
ONSET_THRESHOLD = 0.3
MIN_DELAY = 0.5

# Načtení modelů
svm_filter = load('kalimba_filter.joblib')
ocsvm = svm_filter['model']
scaler = svm_filter['scaler']
cnn_model = tf.keras.models.load_model("tone_recognition_multi_label.h5")

label_to_tone = {
    0: "C4", 1: "D4", 2: "E4", 3: "F4", 4: "G4",
    5: "A4", 6: "B4", 7: "C5", 8: "D5", 9: "E5",
    10: "F5", 11: "G5", 12: "A5", 13: "B5", 
    14: "C6", 15: "D6", 16: "E6"
}

audio_queue = Queue()
last_detection_time = 0

def audio_callback(indata, frames, time, status):
    audio_queue.put(indata.copy())

def extract_features(buffer):
    if len(buffer) < 512:
        return None
    
    try:
        # Základní kontrola energie signálu
        if librosa.feature.rms(y=buffer).mean() < 0.001:
            return None
            
        mfcc = librosa.feature.mfcc(y=buffer, sr=SAMPLE_RATE, n_mfcc=20)
        mfcc_mean = np.mean(mfcc.T, axis=0)
        
        # Výpočet chroma s explicitními parametry
        chroma = librosa.feature.chroma_stft(
            y=buffer, 
            sr=SAMPLE_RATE,
            n_fft=2048,
            hop_length=512,
            tuning=0.0  # Explicitně zakážeme automatické ladění
        )
        
        sc = np.mean(librosa.feature.spectral_centroid(y=buffer, sr=SAMPLE_RATE))
        zcr = np.mean(librosa.feature.zero_crossing_rate(buffer))
        
        return np.concatenate([mfcc_mean, [sc, zcr, np.mean(chroma)]])
    except Exception as e:
        print(f"Feature extraction error: {str(e)}")
        return None

def is_kalimba(buffer, threshold=-0.5):
    if len(buffer) < MIN_SEGMENT_LENGTH:
        return False
    
    features = extract_features(buffer)
    if features is None or features.shape[0] != 23:  # 20 MFCC + 3 features
        return False
    
    try:
        scaled = scaler.transform([features])
        return ocsvm.decision_function(scaled) > threshold
    except:
        return False

def process_audio(buffer):
    if len(buffer) < MIN_SEGMENT_LENGTH or not is_kalimba(buffer):
        return None, 0
    
    try:
        mel = librosa.feature.melspectrogram(
            y=buffer, sr=SAMPLE_RATE, n_fft=2048, hop_length=512, n_mels=128
        )
        mel_db = librosa.power_to_db(mel, ref=np.max)
        mel_db = np.expand_dims(np.expand_dims(mel_db, -1), 0)
        mel_db = tf.image.resize(mel_db, (128, 128)).numpy()
        preds = cnn_model.predict(mel_db, verbose=0)
        return np.argmax(preds[0]), np.max(preds[0])
    except:
        return None, 0

def detect_onset(buffer):
    try:
        return librosa.onset.onset_detect(
            y=buffer, sr=SAMPLE_RATE, units='samples', hop_length=512,
            delta=ONSET_THRESHOLD, pre_max=3, post_max=3
        )
    except:
        return []

with sd.InputStream(
    callback=audio_callback,
    channels=1,
    samplerate=SAMPLE_RATE,
    blocksize=int(SAMPLE_RATE * 0.1)
):
    print("=== KALIMBA DETECTOR ===")
    print("Hrajte tóny... (Ctrl+C pro ukončení)\n")
    
    buffer = np.array([], dtype=np.float32)
    
    while True:
        # Aktualizace bufferu
        while not audio_queue.empty():
            buffer = np.concatenate([buffer, audio_queue.get().squeeze()])
        
        max_length = int(SAMPLE_RATE * BUFFER_DURATION)
        buffer = buffer[-max_length:] if len(buffer) > max_length else buffer
        
        # Hlavní detekce
        if len(buffer) >= MIN_SEGMENT_LENGTH:
            onsets = detect_onset(buffer)
            current_time = time.time()
            
            if len(onsets) > 0 and (current_time - last_detection_time) > MIN_DELAY:
                start = max(onsets[-1], 0)
                end = start + MIN_SEGMENT_LENGTH
                
                if end <= len(buffer):
                    segment = buffer[start:end]
                    label, conf = process_audio(segment)
                    
                    if label is not None and conf > 0.5:
                        tone = label_to_tone.get(label, "Neznámý")
                        print(f"{tone} ({conf*100:.1f}%)".ljust(20) + f"| {time.strftime('%H:%M:%S')}\r", end='')
                        last_detection_time = current_time


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\Dave\AppData\Local\Programs\Python\Python311\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\Dave\AppData\Local\Programs\Python\Python311\Lib\site-packages\traitlets\config\application.py", line 1043, in launch_instance
    app.start()
  File "C:\Users\Dave\AppData\Local\Programs\Python\Python311\Lib\site-packages\ipykernel\kernelapp.py", lin

AttributeError: _ARRAY_API not found

ImportError: numpy.core.multiarray failed to import

In [32]:
#GPT

In [ ]:
import numpy as np
import librosa
import tensorflow as tf
import sounddevice as sd
from queue import Queue
import time

# Nastavení parametrů
SAMPLE_RATE = 22050
BUFFER_DURATION = 1.0
ONSET_THRESHOLD = 0.3
MIN_DELAY = 0.5  # Minimální prodleva mezi detekcemi (v sekundách)

# Načtení modelu
model = tf.keras.models.load_model("tone_recognition_multi_label.h5")

audio_queue = Queue()
last_detection_time = 0
label_to_tone = label_to_tone = {
    0: "C4", 
    1: "D4", 
    2: "E4", 
    3: "F4", 
    4: "G4", 
    5: "A4", 
    6: "B4", 
    7: "C5", 
    8: "D5", 
    9: "E5", 
    10: "F5", 
    11: "G5", 
    12: "A5", 
    13: "B5", 
    14: "C6", 
    15: "D6", 
    16: "E6"
}

def audio_callback(indata, frames, time, status):
    audio_queue.put(indata.copy())

def process_audio(buffer):
    mel_spec = librosa.feature.melspectrogram(y=buffer, sr=SAMPLE_RATE, n_fft=2048, hop_length=512, n_mels=128)
    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)
    mel_spec_db = np.expand_dims(mel_spec_db, axis=-1)
    mel_spec_db = tf.image.resize(mel_spec_db, (128, 128)).numpy()
    input_data = np.expand_dims(mel_spec_db, axis=0)
    predictions = model.predict(input_data, verbose=0)
    return np.argmax(predictions[0]), np.max(predictions[0])

def detect_onset(buffer):
    return librosa.onset.onset_detect(
        y=buffer, sr=SAMPLE_RATE, units='samples', hop_length=512,
        delta=ONSET_THRESHOLD, pre_max=3, post_max=3
    )

with sd.InputStream(callback=audio_callback, channels=1, samplerate=SAMPLE_RATE, blocksize=int(SAMPLE_RATE*0.1)):
    print("=== REALTIME DETEKCE TÓNŮ ===")
    print("Hrajte tón na mikrofon... (Ctrl+C pro ukončení)\n")
    print("Poslední detekce:".ljust(40) + "Čekám na tón...\r", end='')
    
    buffer = np.array([], dtype=np.float32)
    
    while True:
        # Aktualizace bufferu
        while not audio_queue.empty():
            buffer = np.concatenate([buffer, audio_queue.get().squeeze()])
        
        # Omezíme délku bufferu
        buffer = buffer[-int(SAMPLE_RATE*BUFFER_DURATION):] if len(buffer) > SAMPLE_RATE*BUFFER_DURATION else buffer
        
        # Detekce onsetů
        onsets = detect_onset(buffer)
        current_time = time.time()
        
        if len(onsets) > 0 and (current_time - last_detection_time) > MIN_DELAY:
            start_sample = onsets[-1]
            segment = buffer[start_sample:]
            
            if len(segment) > 2048:
                predicted_label, confidence = process_audio(segment)
                tone = label_to_tone[predicted_label]
                output = f"{tone} ({confidence*100:.1f}%)".ljust(20)
                print(f"Poslední detekce: {output} | Čas: {time.strftime('%H:%M:%S')}\r", end='')
                last_detection_time = current_time

In [46]:
import numpy as np
import librosa
import tensorflow as tf
import sounddevice as sd
import joblib
from queue import Queue
import time

# Nastavení parametrů
SAMPLE_RATE = 22050
BUFFER_DURATION = 1.0
ONSET_THRESHOLD = 0.3
MIN_DELAY = 0.5  # Minimální prodleva mezi detekcemi (v sekundách)

# Načtení modelů
tone_model = tf.keras.models.load_model("tone_recognition_multi_label.h5")
svm_model = joblib.load("kalimba_ocsvm.pkl")
scaler = joblib.load("scaler.pkl")

# Labely tónů
label_to_tone = {
    0: "C4", 1: "D4", 2: "E4", 3: "F4", 4: "G4", 5: "A4", 6: "B4",
    7: "C5", 8: "D5", 9: "E5", 10: "F5", 11: "G5", 12: "A5", 13: "B5",
    14: "C6", 15: "D6", 16: "E6"
}

audio_queue = Queue()
last_detection_time = 0
last_detected_tone = "Čekám na tón..."  # Uchovává poslední validní tón

def audio_callback(indata, frames, time, status):
    audio_queue.put(indata.copy())

# Funkce pro extrakci MFCC příznaků pro SVM
def extract_svm_features(audio_segment):
    mfccs = librosa.feature.mfcc(y=audio_segment, sr=SAMPLE_RATE, n_mfcc=20)
    return np.mean(mfccs, axis=1)  # Průměr přes časovou osu

# Funkce pro extrakci mel spektrogramu pro CNN model
def extract_mel_spectrogram(audio_segment):
    mel_spec = librosa.feature.melspectrogram(y=audio_segment, sr=SAMPLE_RATE, n_fft=2048, hop_length=512, n_mels=128)
    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)
    mel_spec_db = np.expand_dims(mel_spec_db, axis=-1)
    mel_spec_db = tf.image.resize(mel_spec_db, (128, 128)).numpy()
    return np.expand_dims(mel_spec_db, axis=0)

# Funkce pro detekci onsetů (začátků tónů)
def detect_onset(buffer):
    return librosa.onset.onset_detect(
        y=buffer, sr=SAMPLE_RATE, units='samples', hop_length=512,
        delta=ONSET_THRESHOLD, pre_max=3, post_max=3
    )

with sd.InputStream(callback=audio_callback, channels=1, samplerate=SAMPLE_RATE, blocksize=int(SAMPLE_RATE*0.1)):
    print("=== REALTIME DETEKCE TÓNŮ ===")
    print("Hrajte tón na mikrofon... (Ctrl+C pro ukončení)\n")
    print(f"Poslední detekce: {last_detected_tone}".ljust(40), end='\r')
    
    buffer = np.array([], dtype=np.float32)
    
    while True:
        # Aktualizace bufferu
        while not audio_queue.empty():
            buffer = np.concatenate([buffer, audio_queue.get().squeeze()])
        
        # Omezíme délku bufferu
        buffer = buffer[-int(SAMPLE_RATE*BUFFER_DURATION):] if len(buffer) > SAMPLE_RATE*BUFFER_DURATION else buffer
        
        # Detekce onsetů
        onsets = detect_onset(buffer)
        
        current_time = time.time()
        
        if len(onsets) > 0 and (current_time - last_detection_time) > MIN_DELAY:
            start_sample = onsets[-1]
            segment = buffer[start_sample:]
            
            if len(segment) > 2048:  # Zajistíme, že segment má dostatek dat
                
                # **1) Ověření pomocí One-Class SVM**
                svm_features = extract_svm_features(segment)
                svm_features_scaled = scaler.transform([svm_features])
                svm_prediction = svm_model.predict(svm_features_scaled)

                print(f"SVM predikce: {svm_prediction[0]}")

                if svm_prediction[0] == -1:
                    continue  # Pokud zvuk neodpovídá kalimbě, ignorujeme ho
                
                # **2) Klasifikace konkrétního tónu pomocí CNN**
                input_data = extract_mel_spectrogram(segment)
                predictions = tone_model.predict(input_data, verbose=0)
                
                predicted_label = np.argmax(predictions[0])
                confidence = np.max(predictions[0])
                new_tone = f"{label_to_tone[predicted_label]} ({confidence*100:.1f}%)"

                if new_tone != last_detected_tone:  # Výpis se změní jen při detekci nového tónu
                    last_detected_tone = new_tone
                    print(f"Poslední detekce: {last_detected_tone}".ljust(40), end='\r')

                last_detection_time = current_time


=== REALTIME DETEKCE TÓNŮ ===
Hrajte tón na mikrofon... (Ctrl+C pro ukončení)



C:\Users\Dave\AppData\Local\Programs\Python\Python311\Lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=0
  warnings.warn(


SVM predikce: -1
SVM predikce: -1
SVM predikce: -1
SVM predikce: -1
SVM predikce: -1
SVM predikce: -1
SVM predikce: -1
SVM predikce: -1
SVM predikce: -1
SVM predikce: -1
SVM predikce: -1
SVM predikce: -1
SVM predikce: -1
SVM predikce: -1
SVM predikce: -1
SVM predikce: -1
SVM predikce: -1
SVM predikce: -1
SVM predikce: -1
SVM predikce: -1
SVM predikce: -1
SVM predikce: -1
SVM predikce: -1
SVM predikce: -1
SVM predikce: -1
SVM predikce: -1
SVM predikce: -1
SVM predikce: -1
SVM predikce: -1
SVM predikce: -1
SVM predikce: -1
SVM predikce: -1
SVM predikce: -1
SVM predikce: -1
SVM predikce: -1
SVM predikce: -1
SVM predikce: -1
SVM predikce: -1
SVM predikce: -1
SVM predikce: -1
SVM predikce: -1
SVM predikce: -1
SVM predikce: -1
SVM predikce: -1
SVM predikce: -1
SVM predikce: -1
SVM predikce: -1
SVM predikce: -1
SVM predikce: -1
SVM predikce: -1
SVM predikce: -1
SVM predikce: -1
SVM predikce: -1
SVM predikce: -1
SVM predikce: -1
SVM predikce: -1
SVM predikce: -1
SVM predikce: -1
SVM predikce: 

KeyboardInterrupt: 

In [41]:
import numpy as np
import librosa
import tensorflow as tf
import sounddevice as sd
import joblib
from queue import Queue
import time

# Nastavení parametrů
SAMPLE_RATE = 22050
BUFFER_DURATION = 1.0
ONSET_THRESHOLD = 0.3
MIN_DELAY = 0.5

# Načtení modelů
model = tf.keras.models.load_model("tone_recognition_multi_label.h5")
svm_model = joblib.load("kalimba_ocsvm.pkl")
scaler = joblib.load("scaler.pkl")

audio_queue = Queue()
last_detection_time = 0
last_tone = "Žádná detekce"  # Uchovává poslední platný tón
label_to_tone = {
    0: "C4", 1: "D4", 2: "E4", 3: "F4", 4: "G4", 5: "A4", 6: "B4",
    7: "C5", 8: "D5", 9: "E5", 10: "F5", 11: "G5", 12: "A5", 13: "B5",
    14: "C6", 15: "D6", 16: "E6"
}

def audio_callback(indata, frames, time, status):
    audio_queue.put(indata.copy())

def process_audio(buffer):
    mel_spec = librosa.feature.melspectrogram(y=buffer, sr=SAMPLE_RATE, n_fft=2048, hop_length=512, n_mels=128)
    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)
    mel_spec_db = np.expand_dims(mel_spec_db, axis=-1)
    mel_spec_db = tf.image.resize(mel_spec_db, (128, 128)).numpy()
    input_data = np.expand_dims(mel_spec_db, axis=0)
    predictions = model.predict(input_data, verbose=0)
    return np.argmax(predictions[0]), np.max(predictions[0])

def detect_onset(buffer):
    return librosa.onset.onset_detect(
        y=buffer, sr=SAMPLE_RATE, units='samples', hop_length=512,
        delta=ONSET_THRESHOLD, pre_max=3, post_max=3
    )

def extract_features(audio_data, sr=22050):
    mfccs = librosa.feature.mfcc(y=audio_data, sr=sr, n_mfcc=20)
    return np.mean(mfccs, axis=1)

with sd.InputStream(callback=audio_callback, channels=1, samplerate=SAMPLE_RATE, blocksize=int(SAMPLE_RATE*0.1)):
    print("=== REALTIME DETEKCE TÓNŮ ===")
    print("Hrajte tón na mikrofon... (Ctrl+C pro ukončení)\n")
    print("Poslední detekce:".ljust(40) + "Čekám na tón...\r", end='')
    
    buffer = np.array([], dtype=np.float32)
    
    while True:
        # Aktualizace bufferu
        while not audio_queue.empty():
            buffer = np.concatenate([buffer, audio_queue.get().squeeze()])
        
        # Omezení délky bufferu
        buffer = buffer[-int(SAMPLE_RATE*BUFFER_DURATION):] if len(buffer) > SAMPLE_RATE*BUFFER_DURATION else buffer
        
        # Detekce onsetů
        onsets = detect_onset(buffer)
        current_time = time.time()
        
        if len(onsets) > 0 and (current_time - last_detection_time) > MIN_DELAY:
            start_sample = onsets[-1]
            segment = buffer[start_sample:]
            
            if len(segment) > 2048:
                # Ověření pomocí SVM
                mfcc_features = extract_features(segment, SAMPLE_RATE)
                scaled_features = scaler.transform([mfcc_features])
                is_kalimba = svm_model.predict(scaled_features)[0] == 1
                
                if is_kalimba:
                    # Detekce tónu a aktualizace výpisu
                    predicted_label, confidence = process_audio(segment)
                    tone = label_to_tone[predicted_label]
                    last_tone = f"{tone} ({confidence*100:.1f}%)"
                    output = f"Poslední detekce: {last_tone.ljust(20)} | Čas: {time.strftime('%H:%M:%S')}\r"
                    print(output, end='')
                    last_detection_time = current_time
                else:
                    # Krátkodobá informace o cizím zvuku bez přepsání detekce
                    print(f" [Cizí zvuk ignorován]".ljust(30) + "\r", end='')
                    time.sleep(0.5)  # Krátká pauza pro čitelnost
                    print(f"Poslední detekce: {last_tone.ljust(20)} | Čas: {time.strftime('%H:%M:%S')}\r", end='')

=== REALTIME DETEKCE TÓNŮ ===
Hrajte tón na mikrofon... (Ctrl+C pro ukončení)



C:\Users\Dave\AppData\Local\Programs\Python\Python311\Lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=0
  warnings.warn(


KeyboardInterrupt: 

In [49]:
# Pokračujeme dále 3.3.2025:1:51

In [51]:
import numpy as np
import librosa
import tensorflow as tf
import sounddevice as sd
import joblib
from queue import Queue
import time

# Nastavení parametrů
SAMPLE_RATE = 22050
BUFFER_DURATION = 1.0
ONSET_THRESHOLD = 0.3
MIN_DELAY = 0.5  # Minimální prodleva mezi detekcemi (v sekundách)

# Načtení modelů
kalimba_model = joblib.load("kalimba_ocsvm.pkl")
scaler = joblib.load("scaler.pkl")
tone_model = tf.keras.models.load_model("tone_recognition_multi_label.h5")

audio_queue = Queue()
last_detection_time = 0

label_to_tone = {
    0: "C4", 1: "D4", 2: "E4", 3: "F4", 4: "G4", 5: "A4", 6: "B4",
    7: "C5", 8: "D5", 9: "E5", 10: "F5", 11: "G5", 12: "A5", 13: "B5",
    14: "C6", 15: "D6", 16: "E6"
}

def audio_callback(indata, frames, time, status):
    audio_queue.put(indata.copy())

def extract_features(audio_segment):
    mfccs = librosa.feature.mfcc(y=audio_segment, sr=SAMPLE_RATE, n_mfcc=20)
    return np.mean(mfccs, axis=1)  # Průměr přes časovou osu

def is_kalimba(buffer):
    feature_vector = extract_features(buffer)
    feature_scaled = scaler.transform([feature_vector])
    prediction = kalimba_model.predict(feature_scaled)
    return prediction[0] == 1  # 1 = kalimba, -1 = jiný zvuk

def process_audio(buffer):
    mel_spec = librosa.feature.melspectrogram(y=buffer, sr=SAMPLE_RATE, n_fft=2048, hop_length=512, n_mels=128)
    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)
    mel_spec_db = np.expand_dims(mel_spec_db, axis=-1)
    mel_spec_db = tf.image.resize(mel_spec_db, (128, 128)).numpy()
    input_data = np.expand_dims(mel_spec_db, axis=0)
    predictions = tone_model.predict(input_data, verbose=0)
    return np.argmax(predictions[0]), np.max(predictions[0])

def detect_onset(buffer):
    return librosa.onset.onset_detect(
        y=buffer, sr=SAMPLE_RATE, units='samples', hop_length=512,
        delta=ONSET_THRESHOLD, pre_max=3, post_max=3
    )

with sd.InputStream(callback=audio_callback, channels=1, samplerate=SAMPLE_RATE, blocksize=int(SAMPLE_RATE*0.1)):
    print("=== REALTIME DETEKCE KALIMBY A TÓNŮ ===")
    print("Hrajte tón na mikrofon... (Ctrl+C pro ukončení)")
    
    buffer = np.array([], dtype=np.float32)
    
    while True:
        while not audio_queue.empty():
            buffer = np.concatenate([buffer, audio_queue.get().squeeze()])
        
        buffer = buffer[-int(SAMPLE_RATE*BUFFER_DURATION):] if len(buffer) > SAMPLE_RATE*BUFFER_DURATION else buffer
        
        onsets = detect_onset(buffer)
        current_time = time.time()
        
        if len(onsets) > 0 and (current_time - last_detection_time) > MIN_DELAY:
            start_sample = onsets[-1]
            segment = buffer[start_sample:]
            
            if len(segment) > 2048:
                if is_kalimba(segment):
                    predicted_label, confidence = process_audio(segment)
                    tone = label_to_tone[predicted_label]
                    output = f"{tone} ({confidence*100:.1f}%)".ljust(20)
                else:
                    output = "Nejedná se o kalimbu".ljust(20)
                
                print(f"Poslední detekce: {output} | Čas: {time.strftime('%H:%M:%S')}")
                last_detection_time = current_time


=== REALTIME DETEKCE KALIMBY A TÓNŮ ===
Hrajte tón na mikrofon... (Ctrl+C pro ukončení)


C:\Users\Dave\AppData\Local\Programs\Python\Python311\Lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=0
  warnings.warn(


Poslední detekce: Nejedná se o kalimbu | Čas: 01:52:35
Poslední detekce: Nejedná se o kalimbu | Čas: 01:52:35
Poslední detekce: Nejedná se o kalimbu | Čas: 01:52:36
Poslední detekce: Nejedná se o kalimbu | Čas: 01:52:36
Poslední detekce: Nejedná se o kalimbu | Čas: 01:52:37
Poslední detekce: Nejedná se o kalimbu | Čas: 01:52:37
Poslední detekce: Nejedná se o kalimbu | Čas: 01:52:38
Poslední detekce: Nejedná se o kalimbu | Čas: 01:52:39
Poslední detekce: Nejedná se o kalimbu | Čas: 01:52:39
Poslední detekce: Nejedná se o kalimbu | Čas: 01:52:40
Poslední detekce: Nejedná se o kalimbu | Čas: 01:52:40
Poslední detekce: Nejedná se o kalimbu | Čas: 01:52:41
Poslední detekce: Nejedná se o kalimbu | Čas: 01:52:41
Poslední detekce: Nejedná se o kalimbu | Čas: 01:52:42
Poslední detekce: Nejedná se o kalimbu | Čas: 01:52:42
Poslední detekce: Nejedná se o kalimbu | Čas: 01:52:43
Poslední detekce: Nejedná se o kalimbu | Čas: 01:52:43
Poslední detekce: Nejedná se o kalimbu | Čas: 01:52:44
Poslední d

KeyboardInterrupt: 

In [68]:
import numpy as np
import librosa
import tensorflow as tf
import sounddevice as sd
from queue import Queue
import time
import joblib

# Nastavení parametrů
SAMPLE_RATE = 22050
BUFFER_DURATION = 1.0
ONSET_THRESHOLD = 0.3
MIN_DELAY = 0.5

# Načtení modelů
svm_model = joblib.load("kalimba_ocsvm.pkl")
scaler = joblib.load("scaler.pkl")
tone_model = tf.keras.models.load_model("tone_recognition_multi_label.h5")

audio_queue = Queue()
last_detection_time = 0
label_to_tone = {
    0: "C4", 1: "D4", 2: "E4", 3: "F4", 4: "G4", 
    5: "A4", 6: "B4", 7: "C5", 8: "D5", 9: "E5", 
    10: "F5", 11: "G5", 12: "A5", 13: "B5", 14: "C6", 
    15: "D6", 16: "E6"
}

def audio_callback(indata, frames, time, status):
    audio_queue.put(indata.copy())

def extract_features_from_segment(y):
    # MFCC features (přizpůsobte původní implementaci)
    mfccs = librosa.feature.mfcc(y=y, sr=SAMPLE_RATE, n_mfcc=13, n_fft=2048, hop_length=512)
    mfccs_mean = np.mean(mfccs, axis=1)
    mfccs_std = np.std(mfccs, axis=1)
    
    # Další příznaky (přidejte vlastní podle tréninku modelu)
    spectral_centroid = np.mean(librosa.feature.spectral_centroid(y=y, sr=SAMPLE_RATE))
    zero_crossing = np.mean(librosa.feature.zero_crossing_rate(y))
    
    return np.concatenate([mfccs_mean, mfccs_std, [spectral_centroid, zero_crossing]])

def process_audio(buffer):
    mel_spec = librosa.feature.melspectrogram(y=buffer, sr=SAMPLE_RATE, n_fft=2048, 
                                            hop_length=512, n_mels=128)
    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)
    mel_spec_db = np.expand_dims(mel_spec_db, axis=-1)
    mel_spec_db = tf.image.resize(mel_spec_db, (128, 128)).numpy()
    input_data = np.expand_dims(mel_spec_db, axis=0)
    predictions = tone_model.predict(input_data, verbose=0)
    return np.argmax(predictions[0]), np.max(predictions[0])

def detect_onset(buffer):
    return librosa.onset.onset_detect(
        y=buffer, sr=SAMPLE_RATE, units='samples', 
        hop_length=512, delta=ONSET_THRESHOLD, 
        pre_max=3, post_max=3
    )

with sd.InputStream(callback=audio_callback, channels=1, 
                   samplerate=SAMPLE_RATE, blocksize=int(SAMPLE_RATE*0.1)):
    print("=== REALTIME DETEKCE KALIMBY A TÓNŮ ===")
    print("Hrajte tón na mikrofon... (Ctrl+C pro ukončení)\n")
    
    buffer = np.array([], dtype=np.float32)
    
    while True:
        while not audio_queue.empty():
            buffer = np.concatenate([buffer, audio_queue.get().squeeze()])
        
        buffer = buffer[-int(SAMPLE_RATE*BUFFER_DURATION):] if len(buffer) > SAMPLE_RATE*BUFFER_DURATION else buffer
        
        onsets = detect_onset(buffer)
        current_time = time.time()
        
        if len(onsets) > 0 and (current_time - last_detection_time) > MIN_DELAY:
            start_sample = onsets[-1]
            segment = buffer[start_sample:]
            
            if len(segment) > 2048:
                # Detekce kalimby
                features = extract_features_from_segment(segment)
                features_scaled = scaler.transform([features])
                is_kalimba = svm_model.predict(features_scaled)[0]
                
                if is_kalimba == 1:
                    # Klasifikace tónu
                    label, confidence = process_audio(segment)
                    tone = label_to_tone[label]
                    output = f"{tone} ({confidence*100:.1f}%)".ljust(25)
                    status = f"Kalimba: {output}"
                else:
                    status = "Cizí zvuk - není kalimba".ljust(40)
                
                print(f"Poslední detekce: {status} | Čas: {time.strftime('%H:%M:%S')}\r", end='')
                last_detection_time = current_time

=== REALTIME DETEKCE KALIMBY A TÓNŮ ===
Hrajte tón na mikrofon... (Ctrl+C pro ukončení)



ValueError: X has 28 features, but StandardScaler is expecting 13 features as input.

In [69]:
# SUPER FUNGUJE!!! NEJLEPŠÍ ZATÍM!
import numpy as np
import librosa
import tensorflow as tf
import sounddevice as sd
from queue import Queue
import time

# Nastavení parametrů
SAMPLE_RATE = 22050
BUFFER_DURATION = 1.0
ONSET_THRESHOLD = 0.3
MIN_DELAY = 0.5  # Minimální prodleva mezi detekcemi (v sekundách)
CONFIDENCE_THRESHOLD = 0.7  # Práh jistoty 70 %

# Načtení modelu
model = tf.keras.models.load_model("tone_recognition_multi_label.h5")

audio_queue = Queue()
last_detection_time = 0
label_to_tone = {
    0: "C4", 
    1: "D4", 
    2: "E4", 
    3: "F4", 
    4: "G4", 
    5: "A4", 
    6: "B4", 
    7: "C5", 
    8: "D5", 
    9: "E5", 
    10: "F5", 
    11: "G5", 
    12: "A5", 
    13: "B5", 
    14: "C6", 
    15: "D6", 
    16: "E6"
}

def audio_callback(indata, frames, time, status):
    audio_queue.put(indata.copy())

def process_audio(buffer):
    mel_spec = librosa.feature.melspectrogram(y=buffer, sr=SAMPLE_RATE, n_fft=2048, hop_length=512, n_mels=128)
    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)
    mel_spec_db = np.expand_dims(mel_spec_db, axis=-1)
    mel_spec_db = tf.image.resize(mel_spec_db, (128, 128)).numpy()
    input_data = np.expand_dims(mel_spec_db, axis=0)
    predictions = model.predict(input_data, verbose=0)
    return np.argmax(predictions[0]), np.max(predictions[0])

def detect_onset(buffer):
    return librosa.onset.onset_detect(
        y=buffer, sr=SAMPLE_RATE, units='samples', hop_length=512,
        delta=ONSET_THRESHOLD, pre_max=3, post_max=3
    )

with sd.InputStream(callback=audio_callback, channels=1, samplerate=SAMPLE_RATE, blocksize=int(SAMPLE_RATE*0.1)):
    print("=== REALTIME DETEKCE TÓNŮ ===")
    print("Hrajte tón na mikrofon... (Ctrl+C pro ukončení)\n")
    print("Poslední detekce:".ljust(40) + "Čekám na tón...\r", end='')
    
    buffer = np.array([], dtype=np.float32)
    
    while True:
        # Aktualizace bufferu
        while not audio_queue.empty():
            buffer = np.concatenate([buffer, audio_queue.get().squeeze()])
        
        # Omezíme délku bufferu
        buffer = buffer[-int(SAMPLE_RATE*BUFFER_DURATION):] if len(buffer) > SAMPLE_RATE*BUFFER_DURATION else buffer
        
        # Detekce onsetů
        onsets = detect_onset(buffer)
        current_time = time.time()
        
        if len(onsets) > 0 and (current_time - last_detection_time) > MIN_DELAY:
            start_sample = onsets[-1]
            segment = buffer[start_sample:]
            
            if len(segment) > 2048:
                predicted_label, confidence = process_audio(segment)
                if confidence >= CONFIDENCE_THRESHOLD:  # Podmínka pro jistotu nad 70 %
                    tone = label_to_tone[predicted_label]
                    output = f"{tone} ({confidence*100:.1f}%)".ljust(20)
                    print(f"Poslední detekce: {output} | Čas: {time.strftime('%H:%M:%S')}\r", end='')
                    last_detection_time = current_time

=== REALTIME DETEKCE TÓNŮ ===
Hrajte tón na mikrofon... (Ctrl+C pro ukončení)



C:\Users\Dave\AppData\Local\Programs\Python\Python311\Lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=0
  warnings.warn(



KeyboardInterrupt



In [71]:
import numpy as np
import librosa
import tensorflow as tf
import sounddevice as sd
from queue import Queue
import time

# Nastavení parametrů
SAMPLE_RATE = 22050
BUFFER_DURATION = 1.0
ONSET_THRESHOLD = 0.3
MIN_DELAY = 0.5  # Minimální prodleva mezi detekcemi (v sekundách)
CONFIDENCE_THRESHOLD = 0.8  # Práh jistoty 70 %

# Načtení modelu
model = tf.keras.models.load_model("tone_recognition_multi_label.h5")

audio_queue = Queue()
last_detection_time = 0
label_to_tone = {
    0: "C4", 
    1: "D4", 
    2: "E4", 
    3: "F4", 
    4: "G4", 
    5: "A4", 
    6: "B4", 
    7: "C5", 
    8: "D5", 
    9: "E5", 
    10: "F5", 
    11: "G5", 
    12: "A5", 
    13: "B5", 
    14: "C6", 
    15: "D6", 
    16: "E6"
}

# Definice požadované sekvence tónů (podle indexů v label_to_tone)
TARGET_SEQUENCE = [9, 7, 6, 7, 9, 7, 6, 7, 9, 7, 6, 7, 11, 11, 10, 10, 9]
TOTAL_NOTES = len(TARGET_SEQUENCE)  # Celkový počet tónů v sekvenci (17)
current_position = 0  # Aktuální pozice v sekvenci
correct_notes = 0  # Počet správně zahraných tónů

def audio_callback(indata, frames, time, status):
    audio_queue.put(indata.copy())

def process_audio(buffer):
    mel_spec = librosa.feature.melspectrogram(y=buffer, sr=SAMPLE_RATE, n_fft=2048, hop_length=512, n_mels=128)
    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)
    mel_spec_db = np.expand_dims(mel_spec_db, axis=-1)
    mel_spec_db = tf.image.resize(mel_spec_db, (128, 128)).numpy()
    input_data = np.expand_dims(mel_spec_db, axis=0)
    predictions = model.predict(input_data, verbose=0)
    return np.argmax(predictions[0]), np.max(predictions[0])

def detect_onset(buffer):
    return librosa.onset.onset_detect(
        y=buffer, sr=SAMPLE_RATE, units='samples', hop_length=512,
        delta=ONSET_THRESHOLD, pre_max=3, post_max=3
    )

with sd.InputStream(callback=audio_callback, channels=1, samplerate=SAMPLE_RATE, blocksize=int(SAMPLE_RATE*0.1)):
    print("=== REALTIME DETEKCE TÓNŮ ===")
    print("Hrajte tóny v pořadí: E5 C5 B4 C5 E5 C5 B4 C5 E5 C5 B4 C5 G5 G5 F5 F5 E5")
    print(f"Celkem tónů: {TOTAL_NOTES}\n")
    print("Poslední detekce:".ljust(40) + "Čekám na tón...\r", end='')
    
    buffer = np.array([], dtype=np.float32)
    
    while True:
        # Aktualizace bufferu
        while not audio_queue.empty():
            buffer = np.concatenate([buffer, audio_queue.get().squeeze()])
        
        # Omezíme délku bufferu
        buffer = buffer[-int(SAMPLE_RATE*BUFFER_DURATION):] if len(buffer) > SAMPLE_RATE*BUFFER_DURATION else buffer
        
        # Detekce onsetů
        onsets = detect_onset(buffer)
        current_time = time.time()
        
        if len(onsets) > 0 and (current_time - last_detection_time) > MIN_DELAY:
            start_sample = onsets[-1]
            segment = buffer[start_sample:]
            
            if len(segment) > 2048:
                predicted_label, confidence = process_audio(segment)
                if confidence >= CONFIDENCE_THRESHOLD:  # Podmínka pro jistotu nad 70 %
                    tone = label_to_tone[predicted_label]
                    # Kontrola, zda detekovaný tón odpovídá aktuálnímu v sekvenci
                    if predicted_label == TARGET_SEQUENCE[current_position]:
                        correct_notes += 1
                        current_position += 1
                    else:
                        current_position = 0  # Reset pozice při chybě
                        correct_notes = 0     # Reset správných tónů při chybě
                    
                    # Výpis výsledku
                    output = f"{tone} ({confidence*100:.1f}%)".ljust(20)
                    progress = f"Správně: {correct_notes}/{TOTAL_NOTES}"
                    print(f"Poslední detekce: {output} | {progress} | Čas: {time.strftime('%H:%M:%S')}\r", end='')
                    last_detection_time = current_time
                    
                    # Pokud je sekvence dokončena
                    if current_position == TOTAL_NOTES:
                        print(f"\nGratuluji! Úspěšně jste zahráli celou sekvenci ({correct_notes}/{TOTAL_NOTES})!")
                        break

=== REALTIME DETEKCE TÓNŮ ===
Hrajte tóny v pořadí: E5 C5 B4 C5 E5 C5 B4 C5 E5 C5 B4 C5 G5 G5 F5 F5 E5
Celkem tónů: 17




KeyboardInterrupt



In [5]:
import numpy as np
import librosa
import tensorflow as tf
import sounddevice as sd
from queue import Queue
import time

# Nastavení parametrů
SAMPLE_RATE = 22050
BUFFER_DURATION = 1.0
ONSET_THRESHOLD = 0.3
MIN_DELAY = 2.0  # Minimální prodleva mezi detekcemi (zvýšeno na 1 sekundu)
CONFIDENCE_THRESHOLD = 0.8  # Práh jistoty 70 %

# Načtení modelu
model = tf.keras.models.load_model("tone_recognition_multi_label.h5")

audio_queue = Queue()
last_detection_time = 0
label_to_tone = {
    0: "C4", 
    1: "D4", 
    2: "E4", 
    3: "F4", 
    4: "G4", 
    5: "A4", 
    6: "B4", 
    7: "C5", 
    8: "D5", 
    9: "E5", 
    10: "F5", 
    11: "G5", 
    12: "A5", 
    13: "B5", 
    14: "C6", 
    15: "D6", 
    16: "E6"
}

# Definice požadované sekvence tónů (podle indexů v label_to_tone)
TARGET_SEQUENCE = [9, 7, 6, 7, 9, 7, 6, 7, 9, 7, 6, 7, 11, 11, 10, 10, 9]
TOTAL_NOTES = len(TARGET_SEQUENCE)  # Celkový počet tónů v sekvenci (17)
current_position = 0  # Aktuální pozice v sekvenci
correct_notes = 0  # Počet správně zahraných tónů

def audio_callback(indata, frames, time, status):
    audio_queue.put(indata.copy())

def process_audio(buffer):
    mel_spec = librosa.feature.melspectrogram(y=buffer, sr=SAMPLE_RATE, n_fft=2048, hop_length=512, n_mels=128)
    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)
    mel_spec_db = np.expand_dims(mel_spec_db, axis=-1)
    mel_spec_db = tf.image.resize(mel_spec_db, (128, 128)).numpy()
    input_data = np.expand_dims(mel_spec_db, axis=0)
    predictions = model.predict(input_data, verbose=0)
    return np.argmax(predictions[0]), np.max(predictions[0])

def detect_onset(buffer):
    return librosa.onset.onset_detect(
        y=buffer, sr=SAMPLE_RATE, units='samples', hop_length=512,
        delta=ONSET_THRESHOLD, pre_max=3, post_max=3
    )

with sd.InputStream(callback=audio_callback, channels=1, samplerate=SAMPLE_RATE, blocksize=int(SAMPLE_RATE*0.1)):
    print("=== REALTIME DETEKCE TÓNŮ ===")
    print("Hrajte tóny v pořadí: E5 C5 B4 C5 E5 C5 B4 C5 E5 C5 B4 C5 G5 G5 F5 F5 E5")
    print(f"Celkem tónů: {TOTAL_NOTES}\n")
    print("Poslední detekce:".ljust(40) + "Čekám na tón...\r", end='')
    
    buffer = np.array([], dtype=np.float32)
    
    while True:
        # Aktualizace bufferu
        while not audio_queue.empty():
            buffer = np.concatenate([buffer, audio_queue.get().squeeze()])
        
        # Omezíme délku bufferu
        buffer = buffer[-int(SAMPLE_RATE*BUFFER_DURATION):] if len(buffer) > SAMPLE_RATE*BUFFER_DURATION else buffer
        
        # Detekce onsetů
        onsets = detect_onset(buffer)
        current_time = time.time()
        
        if len(onsets) > 0 and (current_time - last_detection_time) > MIN_DELAY:
            start_sample = onsets[-1]
            segment = buffer[start_sample:]
            
            if len(segment) > 2048:
                predicted_label, confidence = process_audio(segment)
                if confidence >= CONFIDENCE_THRESHOLD:  # Podmínka pro jistotu nad 70 %
                    tone = label_to_tone[predicted_label]
                    # Kontrola, zda detekovaný tón odpovídá aktuálnímu v sekvenci
                    if predicted_label == TARGET_SEQUENCE[current_position]:
                        correct_notes += 1
                        current_position += 1
                    else:
                        current_position = 0  # Reset pozice při chybě
                        correct_notes = 0     # Reset správných tónů při chybě
                    
                    # Výpis výsledku
                    output = f"{tone} ({confidence*100:.1f}%)".ljust(20)
                    progress = f"Správně: {correct_notes}/{TOTAL_NOTES}"
                    print(f"Poslední detekce: {output} | {progress} | Čas: {time.strftime('%H:%M:%S')}\r", end='')
                    last_detection_time = current_time
                    
                    # Pokud je sekvence dokončena
                    if current_position == TOTAL_NOTES:
                        print(f"\nGratuluji! Úspěšně jste zahráli celou sekvenci ({correct_notes}/{TOTAL_NOTES})!")
                        break

=== REALTIME DETEKCE TÓNŮ ===
Hrajte tóny v pořadí: E5 C5 B4 C5 E5 C5 B4 C5 E5 C5 B4 C5 G5 G5 F5 F5 E5
Celkem tónů: 17

Poslední detekce: E6 (83.9%)           | Správně: 0/17 | Čas: 22:41:40

KeyboardInterrupt: 

In [ ]:
import numpy as np
import librosa
import tensorflow as tf
import sounddevice as sd
from queue import Queue
import time

import sys
print(sys.version)

print(np.__version__)

# Nastavení parametrů
SAMPLE_RATE = 22050
BUFFER_DURATION = 1.0
ONSET_THRESHOLD = 0.3
MIN_DELAY = 0.5  # Minimální prodleva mezi detekcemi (v sekundách)
CONFIDENCE_THRESHOLD = 0.7  # Minimální požadovaná jistota detekce

# Načtení modelu
model = tf.keras.models.load_model("tone_recognition_multi_label.h5")

audio_queue = Queue()
last_detection_time = 0
label_to_tone = {
    0: "C4", 1: "D4", 2: "E4", 3: "F4", 4: "G4", 5: "A4", 6: "B4", 
    7: "C5", 8: "D5", 9: "E5", 10: "F5", 11: "G5", 12: "A5", 13: "B5", 
    14: "C6", 15: "D6", 16: "E6"
}

def audio_callback(indata, frames, time, status):
    audio_queue.put(indata.copy())

def process_audio(buffer):
    mel_spec = librosa.feature.melspectrogram(y=buffer, sr=SAMPLE_RATE, n_fft=2048, hop_length=512, n_mels=128)
    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)
    mel_spec_db = np.expand_dims(mel_spec_db, axis=-1)
    mel_spec_db = tf.image.resize(mel_spec_db, (128, 128)).numpy()
    input_data = np.expand_dims(mel_spec_db, axis=0)
    predictions = model.predict(input_data, verbose=0)
    return np.argmax(predictions[0]), np.max(predictions[0])

def detect_onset(buffer):
    return librosa.onset.onset_detect(
        y=buffer, sr=SAMPLE_RATE, units='samples', hop_length=512,
        delta=ONSET_THRESHOLD, pre_max=3, post_max=3
    )

with sd.InputStream(callback=audio_callback, channels=1, samplerate=SAMPLE_RATE, blocksize=int(SAMPLE_RATE*0.1)):
    print("=== REALTIME DETEKCE TÓNŮ ===")
    print("Hrajte tón na mikrofon... (Ctrl+C pro ukončení)\n")
    print("Poslední detekce:".ljust(40) + "Čekám na tón...\r", end='')
    
    buffer = np.array([], dtype=np.float32)
    
    while True:
        # Aktualizace bufferu
        while not audio_queue.empty():
            buffer = np.concatenate([buffer, audio_queue.get().squeeze()])
        
        # Omezíme délku bufferu
        buffer = buffer[-int(SAMPLE_RATE*BUFFER_DURATION):] if len(buffer) > SAMPLE_RATE*BUFFER_DURATION else buffer
        
        # Detekce onsetů
        onsets = detect_onset(buffer)
        current_time = time.time()
        
        if len(onsets) > 0 and (current_time - last_detection_time) > MIN_DELAY:
            start_sample = onsets[-1]
            segment = buffer[start_sample:]
            
            if len(segment) > 2048:
                predicted_label, confidence = process_audio(segment)
                if confidence >= CONFIDENCE_THRESHOLD:
                    tone = label_to_tone[predicted_label]
                    output = f"{tone} ({confidence*100:.1f}%)".ljust(20)
                    print(f"Poslední detekce: {output} | Čas: {time.strftime('%H:%M:%S')}\r", end='')
                    last_detection_time = current_time


3.12.9 (tags/v3.12.9:fdb8142, Feb  4 2025, 15:27:58) [MSC v.1942 64 bit (AMD64)]
2.0.0


c:\Users\Dave\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


=== REALTIME DETEKCE TÓNŮ ===
Hrajte tón na mikrofon... (Ctrl+C pro ukončení)



c:\Users\Dave\AppData\Local\Programs\Python\Python312\Lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=0
  warnings.warn(
